# Load Dataset

In [1]:
!pip install -q datasets
!pip install dahuffman
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import ViTModel
import numpy as np
import torch
import cv2
import torch.nn as nn
from transformers import ViTModel, ViTConfig
from torchvision import transforms
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import torch.nn as nn
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import numpy as np
from collections import Counter
import torch
from collections import Counter
from dahuffman import HuffmanCodec



In [3]:
dataset = load_dataset("Matthijs/snacks")
print(dataset)
print(dataset["train"].features['label'].names)

labels = dataset["train"].features["label"].names
num_labels = len(dataset["train"].features["label"].names)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
print(label2id)
print(id2label)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4838
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 952
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 955
    })
})
['apple', 'banana', 'cake', 'candy', 'carrot', 'cookie', 'doughnut', 'grape', 'hot dog', 'ice cream', 'juice', 'muffin', 'orange', 'pineapple', 'popcorn', 'pretzel', 'salad', 'strawberry', 'waffle', 'watermelon']
{'apple': 0, 'banana': 1, 'cake': 2, 'candy': 3, 'carrot': 4, 'cookie': 5, 'doughnut': 6, 'grape': 7, 'hot dog': 8, 'ice cream': 9, 'juice': 10, 'muffin': 11, 'orange': 12, 'pineapple': 13, 'popcorn': 14, 'pretzel': 15, 'salad': 16, 'strawberry': 17, 'waffle': 18, 'watermelon': 19}
{0: 'apple', 1: 'banana', 2: 'cake', 3: 'candy', 4: 'carrot', 5: 'cookie', 6: 'doughnut', 7: 'grape', 8: 'hot dog', 9: 'ice cream', 10: 'juice', 11: 'muffin', 12: 'orange', 13: 'pineapple', 14: 'popcorn', 15: 'pretzel'

# Forward Pass ViT Model with a Toy Image

In [4]:
# Create toy image with dim (batch x channel x width x height)
toy_img = torch.rand(1, 3, 48, 48)

# Define conv layer parameters
num_channels = 3
hidden_size = 768
patch_size = 16

projection = nn.Conv2d(num_channels, hidden_size, kernel_size=patch_size,
             stride=patch_size)

out_projection = projection(toy_img)

print(f'Original image size: {toy_img.size()}')
print(f'Size after projection: {out_projection.size()}')


patch_embeddings = out_projection.flatten(2).transpose(1, 2)
print(f'Patch embedding size: {patch_embeddings.size()}')

# Define [CLS] token embedding with the same emb dimension as the patches
batch_size = 1
cls_token = nn.Parameter(torch.randn(1, 1, hidden_size))
cls_tokens = cls_token.expand(batch_size, -1, -1)

# Prepend [CLS] token in the beginning of patch embedding
patch_embeddings = torch.cat((cls_tokens, patch_embeddings), dim=1)
print(f'Patch embedding size: {patch_embeddings.size()}')

# Define position embedding with the same dimension as the patch embedding
position_embeddings = nn.Parameter(torch.randn(batch_size, 10, hidden_size))

# Add position embedding into patch embedding
input_embeddings = patch_embeddings + position_embeddings
print(f'Input embedding size: {input_embeddings.size()}')


# Define parameters for ViT-base (example)
num_heads = 12
num_layers = 12

# Define Transformer encoders' stack
transformer_encoder_layer = nn.TransformerEncoderLayer(
           d_model=hidden_size, nhead=num_heads,
           dim_feedforward=int(hidden_size * 4),
           dropout=0.1)
transformer_encoder = nn.TransformerEncoder(
           encoder_layer=transformer_encoder_layer,
           num_layers=num_layers)

# Forward pass
output_embeddings = transformer_encoder(input_embeddings)
print(f' Output embedding size: {output_embeddings.size()}')

# Load pretrained model
model_checkpoint = 'google/vit-base-patch16-224-in21k'
model = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)

# Example input image
input_img = torch.rand(batch_size, num_channels, 224, 224)

# Forward pass input image
output_embedding = model(input_img)
print(output_embedding)
print(f"Ouput embedding size: {output_embedding['last_hidden_state'].size()}")

num_labels = 20

# Define linear classifier layer
classifier = nn.Linear(hidden_size, num_labels)

# Forward pass on the output embedding of [CLS] token
output_classification = classifier(output_embedding['last_hidden_state'][:, 0, :])
print(f"Output embedding size: {output_classification.size()}")

Original image size: torch.Size([1, 3, 48, 48])
Size after projection: torch.Size([1, 768, 3, 3])
Patch embedding size: torch.Size([1, 9, 768])
Patch embedding size: torch.Size([1, 10, 768])
Input embedding size: torch.Size([1, 10, 768])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


 Output embedding size: torch.Size([1, 10, 768])


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.0168, -0.2364,  0.0634,  ...,  0.2170,  0.0391, -0.3909],
         [ 0.1858, -0.0258,  0.0103,  ...,  0.0031,  0.2379, -0.2486],
         [-0.0126, -0.1794,  0.0039,  ...,  0.2000,  0.1049, -0.3877],
         ...,
         [ 0.0593, -0.3082, -0.0706,  ...,  0.1018,  0.1635, -0.3935],
         [ 0.0494, -0.1926, -0.1067,  ...,  0.0886,  0.2022, -0.4511],
         [ 0.1409, -0.0764, -0.0165,  ..., -0.0714,  0.0905, -0.2824]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=None, hidden_states=None, attentions=None)
Ouput embedding size: torch.Size([1, 197, 768])
Output embedding size: torch.Size([1, 20])


# Fine Tuning ViT Impelementation

In [5]:
#Pretrained model checkpoint
model_checkpoint = 'google/vit-base-patch16-224-in21k'

In [6]:
class ImageDataset(torch.utils.data.Dataset):

  def __init__(self, input_data):

      self.input_data = input_data
      self.transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224), antialias=True),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
        ])

  def __len__(self):
      return len(self.input_data)

  def get_images(self, idx):
      return self.transform(self.input_data[idx]['image'])

  def get_labels(self, idx):
      return self.input_data[idx]['label']

  def __getitem__(self, idx):
      # Get input data in a batch
      train_images = self.get_images(idx)
      train_labels = self.get_labels(idx)

      return train_images, train_labels

In [7]:
def train_model(model, train_dataset, epochs, learning_rate, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = model.to(device)

    train_loader = DataLoader(ImageDataset(train_dataset), batch_size=batch_size, shuffle=True, num_workers=1)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        total_loss, total_acc = 0, 0

        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

        avg_loss = total_loss / len(train_loader.dataset)
        avg_acc = total_acc / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return model

def test_model(model, test_dataset, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = model.to(device)

    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model.eval()
    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc

In [8]:
import torch
import torch.nn as nn
from transformers import ViTModel, ViTConfig

class ViT(nn.Module):
    def __init__(self, config=ViTConfig(), num_labels=20,
                 model_checkpoint='google/vit-base-patch16-224-in21k',
                 encoded_weights=None, codec=None):
        super(ViT, self).__init__()
        self.vit = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.encoded_weights = encoded_weights
        self.codec = codec

    def forward(self, x):
        x = self.vit(x)['last_hidden_state']
        output = self.classifier(x[:, 0, :])
        return output


In [9]:
# Hyperparameters
EPOCHS = 3
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = ViT().to(device)
trained_model = train_model(model, dataset['train'], EPOCHS, LEARNING_RATE, BATCH_SIZE)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 152/152 [02:35<00:00,  1.02s/it]


Epoch 1/3 - Loss: 0.0452, Accuracy: 0.8233


100%|██████████| 152/152 [02:39<00:00,  1.05s/it]


Epoch 2/3 - Loss: 0.0093, Accuracy: 0.9686


100%|██████████| 152/152 [02:39<00:00,  1.05s/it]

Epoch 3/3 - Loss: 0.0039, Accuracy: 0.9872


In [10]:
test_loss, test_accuracy = test_model(trained_model, dataset['test'], BATCH_SIZE)
print(test_loss, test_accuracy)

100%|██████████| 30/30 [00:11<00:00,  2.66it/s]

Test - Loss: 0.0089, Accuracy: 0.9275
0.00890807471308513 0.9275210084033614


In [11]:
ViT()

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ViT(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072,

In [144]:
trained_model.eval()  # Set the model to evaluation mode
model.to('cpu')  # Move the model to CPU

# Print weights of a specific layer before quantization
# Example: Printing weights of the first linear layer in the classifier
print("Weights before quantization:")
print(trained_model.classifier.weight.data)

# Apply dynamic quantization to the entire model
quantized_model = torch.quantization.quantize_dynamic(
    trained_model,  # the original model
    dtype=torch.qint8  # the data type to convert to (qint8 is typically used for dynamic quantization)
)

# Print weights of the corresponding layer after quantization
# Note: Quantized weights are stored differently, so we use .dequantize() to view them
# print("\nWeights after quantization:")
# print(quantized_model.classifier.weight().dequantize().data)

Weights before quantization:
tensor([[-0.0298, -0.0352, -0.0410,  ...,  0.0474, -0.0362,  0.0482],
        [-0.0228, -0.0166, -0.0078,  ..., -0.0246,  0.0135,  0.0011],
        [-0.0055,  0.0412,  0.0393,  ...,  0.0532,  0.0186, -0.0161],
        ...,
        [ 0.0043,  0.0427, -0.0237,  ..., -0.0433,  0.0211, -0.0212],
        [ 0.0076, -0.0460,  0.0417,  ...,  0.0482, -0.0317, -0.0393],
        [ 0.0021,  0.0208, -0.0263,  ..., -0.0499, -0.0277,  0.0248]])


In [145]:
def test_model_quantized(model, test_dataset, batch_size):

    use_cuda = torch.cuda.is_available()
    #device = torch.device("cuda" if use_cuda else "cpu")

    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model.eval()
    model.to('cpu')

    criterion = nn.CrossEntropyLoss().to("cpu")

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to("cpu"), labels.to("cpu")
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc

# Test the model
test_loss, test_accuracy = test_model_quantized(quantized_model, dataset['test'], BATCH_SIZE)
print(test_loss, test_accuracy)

100%|██████████| 30/30 [01:51<00:00,  3.71s/it]

Test - Loss: 0.0099, Accuracy: 0.9202
0.00988980322027532 0.9201680672268907


In [146]:
import torch
import os

# Assuming 'model' is your original ViT model and 'quantized_model' is the quantized version

# Save the original model
torch.save(trained_model.state_dict(), 'original_model.pth')

# Save the quantized model
torch.save(quantized_model.state_dict(), 'quantized_model.pth')

# Function to get file size in MB
def file_size_in_mb(file_path):
    size_bytes = os.path.getsize(file_path)
    return size_bytes / (1024 * 1024)  # Convert bytes to megabytes

# Print the file sizes
original_size = file_size_in_mb('original_model.pth')
quantized_size = file_size_in_mb('quantized_model.pth')
print(f"Original Model Size: {original_size:.2f} MB")
print(f"Quantized Model Size: {quantized_size:.2f} MB")


Original Model Size: 327.43 MB
Quantized Model Size: 84.44 MB


In [148]:
encoded_weights_dict = {}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
quantized_model.to(device)

precision = 2  # Lowest we can go is 2.
all_data = []

all_weights = [p.data.flatten().tolist() for _, p in quantized_model.named_parameters()]
all_weights_flat = [round(w, precision) for weights in all_weights for w in weights]

# Include specific layers if they are not in named_parameters
specific_layers = [
    "vit.encoder.layer.0.attention.attention.query.weight",
"vit.encoder.layer.0.attention.attention.query.bias",

"vit.encoder.layer.0.attention.attention.key.weight",

"vit.encoder.layer.0.attention.attention.key.bias",

"vit.encoder.layer.0.attention.attention.value.weight",

"vit.encoder.layer.0.attention.attention.value.bias",

"vit.encoder.layer.0.attention.output.dense.weight",

"vit.encoder.layer.0.attention.output.dense.bias",

"vit.encoder.layer.0.intermediate.dense.weight",

"vit.encoder.layer.0.intermediate.dense.bias",

"vit.encoder.layer.0.output.dense.weight",

"vit.encoder.layer.0.output.dense.bias",

"vit.encoder.layer.1.attention.attention.query.weight",

"vit.encoder.layer.1.attention.attention.query.bias",

"vit.encoder.layer.1.attention.attention.key.weight",

"vit.encoder.layer.1.attention.attention.key.bias",

"vit.encoder.layer.1.attention.attention.value.weight",

"vit.encoder.layer.1.attention.attention.value.bias",

"vit.encoder.layer.1.attention.output.dense.weight",

"vit.encoder.layer.1.attention.output.dense.bias",

"vit.encoder.layer.1.intermediate.dense.weight",

"vit.encoder.layer.1.intermediate.dense.bias",

"vit.encoder.layer.1.output.dense.weight",

"vit.encoder.layer.1.output.dense.bias",

"vit.encoder.layer.2.attention.attention.query.weight",

"vit.encoder.layer.2.attention.attention.query.bias",

"vit.encoder.layer.2.attention.attention.key.weight",

"vit.encoder.layer.2.attention.attention.key.bias",

"vit.encoder.layer.2.attention.attention.value.weight",

"vit.encoder.layer.2.attention.attention.value.bias",

"vit.encoder.layer.2.attention.output.dense.weight",

"vit.encoder.layer.2.attention.output.dense.bias",

"vit.encoder.layer.2.intermediate.dense.weight",

"vit.encoder.layer.2.intermediate.dense.bias",

"vit.encoder.layer.2.output.dense.weight",

"vit.encoder.layer.2.output.dense.bias",


"vit.encoder.layer.3.attention.attention.query.weight",

"vit.encoder.layer.3.attention.attention.query.bias",

"vit.encoder.layer.3.attention.attention.key.weight",

"vit.encoder.layer.3.attention.attention.key.bias",

"vit.encoder.layer.3.attention.attention.value.weight",

"vit.encoder.layer.3.attention.attention.value.bias",

"vit.encoder.layer.3.attention.output.dense.weight",

"vit.encoder.layer.3.attention.output.dense.bias",

"vit.encoder.layer.3.intermediate.dense.weight",

"vit.encoder.layer.3.intermediate.dense.bias",

"vit.encoder.layer.3.output.dense.weight",

"vit.encoder.layer.3.output.dense.bias",


"vit.encoder.layer.4.attention.attention.query.weight",

"vit.encoder.layer.4.attention.attention.query.bias",

"vit.encoder.layer.4.attention.attention.key.weight",

"vit.encoder.layer.4.attention.attention.key.bias",

"vit.encoder.layer.4.attention.attention.value.weight",

"vit.encoder.layer.4.attention.attention.value.bias",

"vit.encoder.layer.4.attention.output.dense.weight",

"vit.encoder.layer.4.attention.output.dense.bias",

"vit.encoder.layer.4.intermediate.dense.weight",

"vit.encoder.layer.4.intermediate.dense.bias",

"vit.encoder.layer.4.output.dense.weight",

"vit.encoder.layer.4.output.dense.bias",


"vit.encoder.layer.5.attention.attention.query.weight",

"vit.encoder.layer.5.attention.attention.query.bias",

"vit.encoder.layer.5.attention.attention.key.weight",

"vit.encoder.layer.5.attention.attention.key.bias",

"vit.encoder.layer.5.attention.attention.value.weight",

"vit.encoder.layer.5.attention.attention.value.bias",

"vit.encoder.layer.5.attention.output.dense.weight",

"vit.encoder.layer.5.attention.output.dense.bias",

"vit.encoder.layer.5.intermediate.dense.weight",

"vit.encoder.layer.5.intermediate.dense.bias",

"vit.encoder.layer.5.output.dense.weight",

"vit.encoder.layer.5.output.dense.bias",


"vit.encoder.layer.6.attention.attention.query.weight",

"vit.encoder.layer.6.attention.attention.query.bias",

"vit.encoder.layer.6.attention.attention.key.weight",

"vit.encoder.layer.6.attention.attention.key.bias",

"vit.encoder.layer.6.attention.attention.value.weight",

"vit.encoder.layer.6.attention.attention.value.bias",

"vit.encoder.layer.6.attention.output.dense.weight",

"vit.encoder.layer.6.attention.output.dense.bias",

"vit.encoder.layer.6.intermediate.dense.weight",

"vit.encoder.layer.6.intermediate.dense.bias",

"vit.encoder.layer.6.output.dense.weight",

"vit.encoder.layer.6.output.dense.bias",


"vit.encoder.layer.7.attention.attention.query.weight",

"vit.encoder.layer.7.attention.attention.query.bias",

"vit.encoder.layer.7.attention.attention.key.weight",

"vit.encoder.layer.7.attention.attention.key.bias",

"vit.encoder.layer.7.attention.attention.value.weight",

"vit.encoder.layer.7.attention.attention.value.bias",

"vit.encoder.layer.7.attention.output.dense.weight",

"vit.encoder.layer.7.attention.output.dense.bias",

"vit.encoder.layer.7.intermediate.dense.weight",

"vit.encoder.layer.7.intermediate.dense.bias",

"vit.encoder.layer.7.output.dense.weight",

"vit.encoder.layer.7.output.dense.bias",


"vit.encoder.layer.8.attention.attention.query.weight",

"vit.encoder.layer.8.attention.attention.query.bias",

"vit.encoder.layer.8.attention.attention.key.weight",

"vit.encoder.layer.8.attention.attention.key.bias",

"vit.encoder.layer.8.attention.attention.value.weight",

"vit.encoder.layer.8.attention.attention.value.bias",

"vit.encoder.layer.8.attention.output.dense.weight",

"vit.encoder.layer.8.attention.output.dense.bias",

"vit.encoder.layer.8.intermediate.dense.weight",

"vit.encoder.layer.8.intermediate.dense.bias",

"vit.encoder.layer.8.output.dense.weight",

"vit.encoder.layer.8.output.dense.bias",


"vit.encoder.layer.9.attention.attention.query.weight",

"vit.encoder.layer.9.attention.attention.query.bias",

"vit.encoder.layer.9.attention.attention.key.weight",

"vit.encoder.layer.9.attention.attention.key.bias",

"vit.encoder.layer.9.attention.attention.value.weight",

"vit.encoder.layer.9.attention.attention.value.bias",

"vit.encoder.layer.9.attention.output.dense.weight",

"vit.encoder.layer.9.attention.output.dense.bias",

"vit.encoder.layer.9.intermediate.dense.weight",

"vit.encoder.layer.9.intermediate.dense.bias",

"vit.encoder.layer.9.output.dense.weight",

"vit.encoder.layer.9.output.dense.bias",


"vit.encoder.layer.10.attention.attention.query.weight",

"vit.encoder.layer.10.attention.attention.query.bias",

"vit.encoder.layer.10.attention.attention.key.weight",

"vit.encoder.layer.10.attention.attention.key.bias",

"vit.encoder.layer.10.attention.attention.value.weight",

"vit.encoder.layer.10.attention.attention.value.bias",

"vit.encoder.layer.10.attention.output.dense.weight",

"vit.encoder.layer.10.attention.output.dense.bias",

"vit.encoder.layer.10.intermediate.dense.weight",

"vit.encoder.layer.10.intermediate.dense.bias",

"vit.encoder.layer.10.output.dense.weight",

"vit.encoder.layer.10.output.dense.bias",


"vit.encoder.layer.11.attention.attention.query.weight",

"vit.encoder.layer.11.attention.attention.query.bias",

"vit.encoder.layer.11.attention.attention.key.weight",

"vit.encoder.layer.11.attention.attention.key.bias",

"vit.encoder.layer.11.attention.attention.value.weight",

"vit.encoder.layer.11.attention.attention.value.bias",

"vit.encoder.layer.11.attention.output.dense.weight",

"vit.encoder.layer.11.attention.output.dense.bias",

"vit.encoder.layer.11.intermediate.dense.weight",

"vit.encoder.layer.11.intermediate.dense.bias",

"vit.encoder.layer.11.output.dense.weight",

"vit.encoder.layer.11.output.dense.bias",

"classifier.weight",

"classifier.bias"
]


from functools import reduce
def get_module_name(layer_name):
  if layer_name.endswith('.weight'):
      return layer_name[:-7]
  elif layer_name.endswith('.bias'):
      return layer_name[:-5]
  else:
      return None

def get_module_by_name(module, access_string):
  names = access_string.split(sep='.')
  return reduce(getattr, names, module)


seen_modules = []

# TODO: ACCESS WEIGHTS (NOT QUANTIZED...) --> CHANGE TO FIND THEM IN QUANTIZED MODEL!!!!!
for layer_name in specific_layers:
    layer = dict(quantized_model.named_parameters()).get(layer_name)
    if layer is not None:
        layer_weights = layer.data.cpu().numpy().flatten().tolist()
        layer_weights = [round(w, precision) for w in layer_weights]
        all_weights_flat.extend(layer_weights)
    else:
      module_name = get_module_name(layer_name)
      if module_name is not None:
        if module_name in seen_modules:
          continue

        module = get_module_by_name(quantized_model,module_name)
        weight, bias = module._packed_params._packed_params.unpack()

        #Add bias
        layer_bias = bias.data.cpu().numpy().flatten().tolist()
        layer_bias = [round(b, precision) for b in layer_bias]
        all_weights_flat.extend(layer_bias)

        #Add weights
        layer_weights = weight.int_repr().cpu().numpy().flatten().tolist() # might not be the best with int_repr
        layer_weights = [round(w, precision) for w in layer_bias]
        all_weights_flat.extend(layer_weights)

        #Add module name to seen modules
        seen_modules.append(module_name)

      else:
          print("layer is found: "+layer_name)




frequencies = Counter(all_weights_flat)
codec = HuffmanCodec.from_frequencies(frequencies)



In [149]:
print(frequencies)

Counter({-0.0: 103756, -0.01: 85405, 0.01: 84348, -0.02: 65148, 0.02: 62530, -0.03: 52095, 0.03: 47964, -0.04: 42730, 0.04: 37531, -0.05: 35683, -0.06: 31126, 0.05: 29643, -0.07: 28023, -0.08: 24501, 0.06: 23520, -0.09: 19177, 0.07: 18230, 0.08: 14181, -0.1: 14009, 0.09: 11167, -0.11: 9679, 0.1: 8767, -0.12: 7108, 0.11: 6959, 0.12: 5515, -0.13: 5158, 0.13: 4388, -0.14: 3662, 0.14: 3429, 0.15: 2899, -0.15: 2882, -0.16: 2379, 0.16: 2233, 0.17: 1965, -0.17: 1844, 0.18: 1631, -0.18: 1591, 0.19: 1389, 0.24: 1370, -0.19: 1330, 0.2: 1282, 0.23: 1200, -0.2: 1153, 0.21: 1132, 0.25: 1046, 0.29: 1014, 0.22: 999, 0.3: 979, 0.28: 979, 0.26: 976, -0.21: 944, 0.27: 932, 0.31: 881, 0.32: 845, 0.33: 811, 0.34: 797, -0.22: 783, 0.35: 723, -0.23: 686, 0.36: 679, 0.37: 634, 0.38: 631, -0.24: 592, 0.4: 572, 0.39: 554, -0.25: 549, 0.41: 473, -0.26: 467, 0.42: 461, 0.45: 460, -0.27: 451, 0.44: 446, -0.28: 426, 0.46: 422, 0.43: 393, -0.29: 372, 0.47: 355, -0.3: 316, -0.31: 314, -0.33: 295, -0.32: 283, -0.34: 

In [199]:
encoded_weights_dict={}

for name, param in quantized_model.named_parameters():
    layer_weights = [round(w.item(), precision) for w in param.data.flatten()]
    encoded_weights = codec.encode(layer_weights)
    encoded_weights_dict[name] = encoded_weights
    decoded_weights = codec.decode(encoded_weights)
    assert layer_weights == decoded_weights, "Mismatch in encoding/decoding!"

In [200]:
print(encoded_weights_dict)

{'vit.embeddings.cls_token': b'_\xc4W\x1a\x95\xf9\xe0@\x00\x12!\xac\x81jHm\x95_\xd2\x92_\xffH\x98\x0cy\xa1\x9fI\x01\xf0\t#\x12^F\xbb\x04@\x00\x12\xf27v>\x97ph 3\xf6\x92\xbfYE\xa4$H\xac$\x90\xf1h._\xd0^\x17\xb2\xf4/\xac\x95\x0fe\xd5\x1c\x95\x9a\x1e\x94\x93$\x02C\xd5\xa0^\xc1+_\x80\x04\x92\t\x85%\xf1\xeb\xd7\xb9\xb1\x0f\x1d\x0b\xfd\x04v\xde\xa8uj\xbft\xa3\xc7\x15\xbdlZ\x1f\xed\xc5?\xeb\xca\xa6\xfd\x83I \xcb\xd9m^K~\x97\xae3\xc5\xf4\x87\xe6\xd1\xfa\xb4\xb7A$2\xac/^=g\xc8=ea)_\xe2K\xf2\x92\xf3\xfbV9\x04)q2_\xd4\x0b\x92\xfd[\xeb\xe4\x99`\xc0J\xd5\xff\xea\xbdz\x08\xb3\x95\xda\xb0\x8e^\xa9Uz\x8e_\xa2^\x9eN\xd2\xef\xaf\x9eZ\xa4\x12\xbf\x98\xf3\xf4\xb7IxIz\xf0\x00\xec\xdeL+Ra|\\\xd2C\xbbE\x02\\r\xac\xa0\x18\x06\t\x01\xbf\xc7\x0c\x12\x1aPL\x85\xa6\x97\xe5\nh\xfa\xf5\x1e\xbe:\xbf\xe5\xb5\xe9\x96$\x97\xa3A \xc8\x7f\xec\x7fP\xf0\xbek\x02\xf4\x80\xc8\x0b\xf1\x05\xe7jk\xd0K\xc4\x91OC\xd3zA\x81\xf8^\xbd\x92J\x0fB\xffK\x8a\xb5IF\x8e\xc7\x85\xff\xea@a\xe4>\xd6\x81\xe5~\x82\xe4E\x11\x86\xfe~\xbc\xd0HR\xe

In [201]:
seen_encoded_modules = []

# TODO: FOR NOW ACCESS WEIGHTS FROM NOT QUANTIZED VERSION --> CHANGE TO FIND THEM IN QUANTIZED MODEL!!!!!
for layer_name in specific_layers:
    layer = dict(quantized_model.named_parameters()).get(layer_name)
    if layer is not None:
        layer_weights = layer.data.cpu().numpy().flatten().tolist()
        layer_weights = [round(w, precision) for w in layer_weights]
        encoded_weights = codec.encode(layer_weights)
        encoded_weights_dict[layer_name] = encoded_weights
    else:
      module_name = get_module_name(layer_name)
      if module_name is not None:
        if module_name in seen_encoded_modules:
          continue

        module = get_module_by_name(quantized_model,module_name)
        weight, bias = module._packed_params._packed_params.unpack()


        #Add weights
        layer_weights = weight.int_repr().cpu().numpy().flatten().tolist() # might not be the best with int_repr
        layer_weights = [round(w, precision) for w in layer_bias]
        encoded_weights = codec.encode(layer_weights)
        encoded_weights_layer = module_name +".weight"

        encoded_weights_dict[encoded_weights_layer] = encoded_weights


        #Add bias
        layer_bias = bias.data.cpu().numpy().flatten().tolist()
        layer_bias = [round(b, precision) for b in layer_bias]
        encoded_bias = codec.encode(layer_bias)
        encoded_bias_layer = module_name+".bias"
        encoded_weights_dict[encoded_bias_layer] = encoded_bias



        #Add module name to seen modules
        seen_encoded_modules.append(module_name)
        print("added module")
      else:
          print("layer is found: "+layer_name)

    decoded_weights = codec.decode(encoded_weights)
    assert layer_weights == decoded_weights, "Mismatch in encoding/decoding!"

print(encoded_weights_dict.keys())
torch.save(encoded_weights_dict, 'encoded_model_weights.pth')

added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
added module
dict_keys(['vit.embeddings.cls_token', 'vit.embeddi

In [202]:
huffman_size = file_size_in_mb('encoded_model_weights.pth')
print(f"Huffman Model Size: {huffman_size:.2f} MB")

Huffman Model Size: 0.86 MB


In [203]:
print(encoded_weights_dict["vit.encoder.layer.9.output.dense.bias"])

b'\xb8dS\x1f\xee8o\x83O\xd6\xf1\xa4\xfew\xcb\xdaz\xd8{I\xb7\xdc\x18=aO\x9f\xec\xe5\xac\xd7\x0b=e\xfc\'\xb9\xaa\xd3x\x07s;{\xa6\x1cT\xe6\xc7\x8cU<\x90\x14\xac#\xdcZ\x10\x92\xd4_K\xee\x87\xa4\xa4\xaaN\xf8\xfdb\xef\x1e\x89\x8d\xa6\x7f^\xde\xe2P_\xfb\xe4\x8b}\x9e\xf6o\xfdz\x90lG\xa6\x92\xf9\xcf\x1a(w\xaf\x928\x99w`\xe9\x15\xdf\xce\xc7\x0cZ<1\xf6\x99\xf2\xd7\x15\xe0\'\x8eI\xe3;\xbc*\xeaL\xc9\xf6\xc9\xeb\xa4F\xce\x87\x1es\xc9\xde\xc4\xb1qZ\xe4d\xd5\xd9\x99$Z\x87s7\x87\xf3|v\xb8y\xc7\x12\xb9\xc3\xbe\xf1/oxf9?\x8d\xa5|v\xa0b\x1d\xe2\xf3\x87Y\xe5\xcd\xb5v\x8a6\xf9 H\x9e\xe0\n\xdbJ\xc4R]n\xdeZ\xb7\x06\xdf\x0e5\xc5\xc6\xa3\x9f\xcb:\x1fe\xf9tw\r\xd1QY\xbb\\o\x95CN\\\xf1o\xe0\xfb\xda\xce\xbcxI\xf9\xfe{[\xdf\xd91\xb4\x10\xd9{O\x0f\xdee\x84\xabm\x07\x93\xf9\xf9\x99\xbd{,\x92C;\x18\xde3\xceBR\xee\x91\x9ep\xeed\xe1sh\xb9\xce\xe6\t\xaa-\xf2Os%`\xc8\xe3\xc3\xc4\xdc\xaf\x94\xfd"\x97\xd3\xb1\x7f\r\xd0\x8b\xae\x16\xd1R\x8b\xe3\xb0\xbf\xe4+\xee~P\x92z\xdfY\xd0.\x88\xbb\xc6\xa3m\x1e\xa5\xbel\xe57stu\x1a\xcf\x

In [204]:
def test_model_huffman(model, test_dataset, batch_size, codec, encoded_weights_file):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model = model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc, model.state_dict()


In [205]:
encoded_weights_dict = torch.load('encoded_model_weights.pth')
model = ViT()
model.to(device)

for name, param in model.named_parameters():
    if name in encoded_weights_dict:
        decoded_weights = codec.decode(encoded_weights_dict[name])
        decoded_weights_tensor = torch.tensor(decoded_weights, device=device).view_as(param.data)
        param.data.copy_(decoded_weights_tensor)
    else:
        print(f"Weight for layer {name} not found in encoded weights.")

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: ignored

In [20]:
model.eval()
test_loss, test_accuracy, state_dic = test_model_huffman(model, dataset['test'], BATCH_SIZE, codec, 'encoded_model_weights.pth')
print(test_loss, test_accuracy)

100%|██████████| 30/30 [00:11<00:00,  2.56it/s]

Test - Loss: 0.0107, Accuracy: 0.9107
0.010699655578917815 0.9107142857142857


In [21]:
# This method is useful if you want to access the parameter by its name as a string
state_dict = quantized_model.state_dict()
bias = state_dict['vit.embeddings.patch_embeddings.projection.weight']
print(bias)

tensor([[[[ 1.4583e-02,  5.0063e-02,  5.4151e-02,  ...,  8.7705e-02,
            1.0772e-01,  1.2023e-01],
          [ 1.2029e-02,  1.3005e-02,  1.8691e-02,  ...,  8.4158e-02,
            3.8800e-02,  8.5884e-02],
          [ 1.6649e-02,  1.0831e-02, -4.8958e-03,  ...,  7.7153e-02,
            3.3585e-02,  3.6088e-02],
          ...,
          [-1.2192e-02, -3.4051e-03, -5.3360e-02,  ..., -5.9245e-02,
           -6.8118e-02,  1.9615e-02],
          [ 4.1196e-02, -4.3512e-02, -2.1702e-02,  ...,  1.0827e-02,
           -4.0232e-02,  8.1864e-02],
          [ 6.9691e-02,  8.1436e-02,  3.8220e-02,  ...,  9.5362e-02,
            1.1562e-01,  1.5380e-01]],

         [[-6.3382e-02, -3.9005e-02, -4.3012e-02,  ..., -5.8777e-02,
           -2.5198e-02, -2.1135e-02],
          [-8.3395e-03, -1.0912e-02, -8.2530e-03,  ..., -3.1720e-02,
           -4.5018e-02, -2.8045e-04],
          [ 4.6433e-02,  3.2979e-02,  1.3393e-02,  ..., -3.4011e-02,
           -3.8735e-02, -3.1597e-02],
          ...,
     

In [22]:
# ANOTHER VERSION
'''
        if self.encoded_weights is not None and self.codec is not None:
                for name, param in self.vit.named_parameters():
                    full_name = "vit." + name
                    if full_name in self.encoded_weights:
                        # Decode the weights
                        decoded_weights = self.codec.decode(self.encoded_weights[full_name])

                        # Convert list to PyTorch tensor
                        decoded_weights_tensor = torch.tensor(decoded_weights, device=param.device)

                        # Reshape if necessary
                        if param.data.size() != decoded_weights_tensor.size():
                            if param.data.numel() == decoded_weights_tensor.numel():
                                # Reshape to match the expected size
                                decoded_weights_tensor = decoded_weights_tensor.view(param.data.size())
                                # Assign the reshaped weights
                                #print(full_name)
                                #print(decoded_weights_tensor)
                            else:
                                raise ValueError(f"Total number of elements mismatch for {full_name}: "
                                                f"expected {param.data.numel()}, got {decoded_weights_tensor.numel()}")

                        param.data = decoded_weights_tensor

    def forward(self, x):
            # Decode and apply weights

            # Forward pass through the model
            x = self.vit(x)['last_hidden_state']
            output = self.classifier(x[:, 0, :])  # Use the embedding of [CLS] token
            return output

'''

'\n        if self.encoded_weights is not None and self.codec is not None:\n                for name, param in self.vit.named_parameters():\n                    full_name = "vit." + name\n                    if full_name in self.encoded_weights:\n                        # Decode the weights\n                        decoded_weights = self.codec.decode(self.encoded_weights[full_name])\n\n                        # Convert list to PyTorch tensor\n                        decoded_weights_tensor = torch.tensor(decoded_weights, device=param.device)\n\n                        # Reshape if necessary\n                        if param.data.size() != decoded_weights_tensor.size():\n                            if param.data.numel() == decoded_weights_tensor.numel():\n                                # Reshape to match the expected size\n                                decoded_weights_tensor = decoded_weights_tensor.view(param.data.size())\n                                # Assign the reshaped weights